# LRU Cache

Design a data structure that follows the constraints of a Least Recently Used (LRU) cache.

Implement the LRUCache class:

LRUCache(int capacity) Initialize the LRU cache with positive size capacity.
int get(int key) Return the value of the key if the key exists, otherwise return -1.
void put(int key, int value) Update the value of the key if the key exists. Otherwise, add the key-value pair to the cache. If the number of keys exceeds the capacity from this operation, evict the least recently used key.
The functions get and put must each run in O(1) average time complexity.

**Example 1:**

Input
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
Output
[null, null, null, 1, null, -1, null, -1, 3, 4]

Explanation
LRUCache lRUCache = new LRUCache(2);
lRUCache.put(1, 1); // cache is {1=1}
lRUCache.put(2, 2); // cache is {1=1, 2=2}
lRUCache.get(1);    // return 1
lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3}
lRUCache.get(2);    // returns -1 (not found)
lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3}
lRUCache.get(1);    // return -1 (not found)
lRUCache.get(3);    // return 3
lRUCache.get(4);    // return 4
 
**Constraints:**

- 1 <= capacity <= 3000
- 0 <= key <= 104
- 0 <= value <= 105
- At most 2 * 105 calls will be made to get and put.

In [1]:
class LRUCache:
    class Node:
        __slots__ = ("k", "v", "prev", "next")
        def __init__(self, k=0, v=0):
            self.k, self.v = k, v
            self.prev = self.next = None

    def __init__(self, capacity: int):
        self.cap = capacity
        self.map = {}  # key -> Node
        # Dummy head/tail to avoid edge cases
        self.head, self.tail = self.Node(), self.Node()
        self.head.next = self.tail
        self.tail.prev = self.head

    # --- internal list helpers ---
    def _add_front(self, node):
        node.prev = self.head
        node.next = self.head.next
        self.head.next.prev = node
        self.head.next = node

    def _remove(self, node):
        p, n = node.prev, node.next
        p.next = n
        n.prev = p

    def _move_to_front(self, node):
        self._remove(node)
        self._add_front(node)

    def _pop_lru(self):
        lru = self.tail.prev
        self._remove(lru)
        return lru

    # --- API ---
    def get(self, key: int) -> int:
        node = self.map.get(key)
        if not node:
            return -1
        self._move_to_front(node)
        return node.v

    def put(self, key: int, value: int) -> None:
        node = self.map.get(key)
        if node:
            node.v = value
            self._move_to_front(node)
            return
        new_node = self.Node(key, value)
        self.map[key] = new_node
        self._add_front(new_node)
        if len(self.map) > self.cap:
            lru = self._pop_lru()
            del self.map[lru.k]